# Budget constraint simulator _child aid policy_ 

data

In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv('ChildAid_parameters.csv')
df.head()

,age,1011_r,11_r,9400_3,0005_3,0506_3,0610_3,1011_3,11_3,9400_2,...,0710_2,1011_2,11_2,9400_1,0005_1,0506_1,0607_1,0710_1,1011_1,11_1
0,0,13000,5000,10000,10000,10000,10000,13000,15000,5000,...,10000,13000,15000,5000,5000,5000,5000,10000,13000,15000
1,1,13000,5000,10000,10000,10000,10000,13000,15000,5000,...,10000,13000,15000,5000,5000,5000,5000,10000,13000,15000
2,2,13000,5000,10000,10000,10000,10000,13000,15000,5000,...,10000,13000,15000,5000,5000,5000,5000,10000,13000,15000
3,3,13000,5000,0,10000,10000,10000,13000,15000,0,...,5000,13000,10000,0,5000,5000,5000,5000,13000,10000
4,4,13000,5000,0,10000,10000,10000,13000,15000,0,...,5000,13000,10000,0,5000,5000,5000,5000,13000,10000


In [28]:
df2 = pd.read_csv('income_rest_parameters.csv')
df2.head()

,num_huyou,income
0,0,622
1,1,660
2,2,698
3,3,736
4,4,774


In [33]:
df2.values[2,1]

698

In [9]:
df3 = pd.read_csv('stardt_date.csv', header = None)
df3

,0
0,19940101
1,20000601
2,20040401
3,20060401
4,20070401
5,20100401
6,20111001
7,20120401


In [26]:
df3.values[0]

array([19940101])

simulate

In [10]:
# implement income restriction

# shotoku-gaku (not shu-nyu-)
# The more one of father's income or mother's one
income = 800

# koujo
# とりあえずずっと同じ控除が続いているとする
# 要素としては、一律、普通障害者、寡婦、勤労学生、特別障害者、寡婦特例
deduction_dummy = np.array([1,0,0,1,0,0])
deduction_element = np.array([8.0, 27.0, 27.0, 27.0, 40.0, 35.0])
deduction = np.dot(deduction_element, deduction_dummy)

# calculate target_income
target_income = income - deduction

# num_huyou
num_huyou = 2

if target_income >= df2.values[num_huyou, 1]:
    rest_dum = 1
else:
    rest_dum = 0

In [11]:
# 上を関数化
# ある年の家計の属性から、翌年に児童手当受給資格を満たすかどうかを判断する関数
# 基本となるdeductionの要素に関するパラメーターはdefaultとして入れておく
# 0が所得制限にかからない、1がかかる

# いまdfを引数にしているところは、年度を引数に取れるようにして、それに対応した所得制限表を取れるようにする。
# つまり、最終的には年度とその年の属性を入れると、よく年に受給資格を満たすか判断する関数を作る
def income_rest(df, income, num_huyou, deduction_dummy, deduction_element = np.array([8.0, 27.0, 27.0, 27.0, 40.0, 35.0])):
    
    target_income = income - np.dot(deduction_element, deduction_dummy)
    
    if target_income >= df.values[num_huyou, 1]:
        rest_dum = 1
    else:
        rest_dum = 0
        
    return rest_dum

In [12]:
income_rest(df2, 340, 2, np.array([1,0,0,1,0,0]))

0

In [23]:
# この関数の中で所得制限にかかるかを判断すりゃいいやんけ
# そうしたら所得制限の閾値を入れたリストさえありゃいいからな
# birth dayは19950120みたいに入れる

changedate = [i[0] for i in df3.values]


def income_rest(df, target_income, num_huyou):
    
    if target_income >= df.values[num_huyou, 1]:
        rest_dum = 1
    else:
        rest_dum = 0
        
    return rest_dum

# dfsは各年のincome restの基準データを格納したdfをリストに入れたもの
def aid_for_onechild(year, birthday, nanshi, num_huyou, target_income, dfs):
    
    birthyear = int(str(birthday)[0:4])
    birthmonth = int(str(birthday)[4:6]) 
    child_aid = 0
    
    age6 = birthyear + 6
    if birthmonth  < 4:
        ent_year = age6
    else:
        ent_year = age6 + 1
    entrance_date = int(str(ent_year) + "0401")
    
    age9 = birthyear + 9
    if birthmonth  < 4:
        third_year = age9
    else:
        third_year = age9 + 1
    third_date = int(str(third_year) + "0401")
    
    age12 = birthyear + 12
    if birthmonth  < 4:
        grad_year = age12
    else:
        grad_year = age12 + 1
    grad_date = int(str(grad_year) + "0401")
    
    age15 = birthyear + 15
    if birthmonth  < 4:
        midle_grad_year = age12
    else:
        midle_grad_year = age15 + 1
    midle_grad_date = int(str(midle_grad_year) + "0401")
    
    for i in ["0101", "0201", "0301", "0401", "0501", "0601", "0701", "0801", "0901", "1001", "1101", "1201"]:
        target_date = int(str(year) + i)
        age = (target_date - birthday)/10000
        period = len([1 for i in changedate if i < target_date])
        
        if period == 1:
            if income_rest(dfs[0], target_income, num_huyou) == 0:
                if age < 3:
                    if nanshi < 3:
                        child_aid += 5000
                    else:
                        child_aid += 10000
                
        if period == 2:
            if income_rest(dfs[1], target_income, num_huyou) == 0:
                if age < 7:
                    # 小学校に入学していないという条件をつける
                    if target_date < entrance_date:
                        if nanshi < 3:
                            child_aid += 5000
                        else:
                            child_aid += 10000
                        
        if period == 3:
            if income_rest(dfs[2], target_income, num_huyou) == 0:
                if age < 10:
                    if target_date < third_date:
                        if nanshi < 3:
                            child_aid += 5000
                        else:
                            child_aid += 10000
                        
        if period == 4:
            if income_rest(dfs[3], target_income, num_huyou) == 0:
                if age < 13:
                    if target_date < grad_date:
                        if nanshi < 3:
                            child_aid += 5000
                        else:
                            child_aid += 10000
                        
        if period == 5:
            if income_rest(dfs[4], target_income, num_huyou) == 0:
                if age < 3:
                    child_aid = 10000
                elif 3 <= age < 13:
                    if target_date < grad_date:
                        if nanshi < 3:
                            child_aid += 5000
                        else:
                            child_aid += 10000
                        
        if period == 6:
            if income_rest(dfs[5], target_income, num_huyou) == 0:
                if age < 16:
                    if target_date < midle_grad_date:
                        child_aid += 13000
        
        # この期は所得制限がないとして出す
        if period == 7:
            if age < 3:
                child_aid = 15000
            else:
                # 小学校終了まで
                if target_date < grad_date:
                    if nanshi < 3:
                        child_aid += 10000
                    else:
                        child_aid += 15000
                else:
                    child_aid += 10000
                    
        if period == 8:
            if income_rest(dfs[6], target_income, num_huyou) == 0:
                if age < 3:
                    child_aid += 15000
                else:
                    # 小学校終了まで
                    if target_date < grad_date:
                        if nanshi < 3:
                            child_aid += 10000
                        else:
                            child_aid += 15000
                    else:
                        child_aid += 10000
            else:
                child_aid += 5000
                    
    return child_aid

# その家計全体での児童手当額
# deductionのところは家計のcharacteristicsに置き換える
def total_aid(year, births, income, num_huyou, deduction_element, deduction_dummy, dfs):
    
    # calculate deduction for this year
    # dudec = calculate_decduction(year, household_characteristics)
    
    deduc = np.dot(deduction_element, deduction_dummy)
    target_income = income - deduc
    
    total_aid = 0
    for n,i in enumerate(births):
        total_aid += aid_for_onechild(year, i, n, num_huyou, target_income, dfs)
    return total_aid

example

In [36]:
# 本当はここを各年のやつにする
dfs = [df2,df2,df2,df2,df2,df2,df2]
child_aid_for_house = total_aid(2003, [19950120, 19990430, 20051113], 400, 4, np.array([8.0, 27.0, 27.0, 27.0, 40.0, 35.0]) ,np.array([1,0,0,1,0,0]), dfs)

In [37]:
child_aid_for_house

120000